In [3]:
import pandas as pd
import numpy as np
import pygtrie as trie
from collections import defaultdict
from scipy.stats import entropy

In [4]:
df = pd.read_csv("village_suffix.txt", sep=' ', header=None, names=["word","cnt","err"])

In [6]:
m = {} # 存储每个候选词的词频
m2 = defaultdict(lambda: defaultdict(int)) # 存储每个候选词的前缀字集合
for index, row in df.iterrows():
    m[row['word']] = row['cnt']
    if len(row['word']) == 1: continue # 单字词没有前缀字，无需计算
    key = row['word'][1:]
    prefix = row['word'][:1]
    m2[key][prefix] += row['cnt']

In [7]:
def poly(x):
    if len(x['word']) == 1 or x['word'][1:] not in m: return 1 # 单字聚合度为1
    return x['cnt']/m[x['word'][1:]]
def flex(x):
    if x['word'] not in m2: return 1
    s = pd.Series(m2[x['word']])
    return entropy(s / s.sum())

In [8]:
df['poly'] = df.apply(poly, axis=1)
df['flex'] = df.apply(flex, axis=1)
df['score'] = df.apply(lambda x: x['flex']*x['poly'], axis=1)

In [9]:
d1 = df[(df['score']>1) & (df['err']/df['cnt']<0.5)].sort_values(['score'], ascending=False)

In [13]:
d1.head(15)

,word,cnt,err,poly,flex,score
5,社区居委会,25049,0,0.993574,5.062190,5.029661
19,社区居民委员会,4533,0,0.996702,4.812210,4.796338
2,村委会,192722,0,0.833641,5.158910,4.300680
13,社区,10209,0,0.808506,5.211187,4.213274
28,嘎查,2260,2,0.999116,3.824331,3.820949
11,村民委员会,15926,0,0.750660,4.544787,3.411589
138,社区村委会,445,1,0.917526,3.431909,3.148865
171,嘎查村,341,2,1.000000,2.794461,2.794461
91,管理区,669,2,1.000000,2.589878,2.589878
300,嘎查村民委员会,180,2,0.994475,2.332284,2.319398
